In [80]:
# !pip install transformers
# !pip install sentencepiece
# !pip install accelerate -U
!pip install sacrebleu
# !pip install unbabel-comet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.0 MB/s eta 0:00:00


In [53]:
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [89]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from sacrebleu.metrics import BLEU, CHRF
from comet import download_model, load_from_checkpoint
from datasets import load_dataset
import time
import warnings
warnings.filterwarnings("ignore")


In [56]:
dataset = load_dataset("wmt14", "de-en", split="validation")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4508785 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3003 [00:00<?, ? examples/s]

Dataset wmt14 downloaded and prepared to /root/.cache/huggingface/datasets/wmt14/de-en/1.0.0/6aa64c5c4f2c1c217718c6d6266aad92d1229e761c57379c53752b8c0e55c93b. Subsequent calls will reuse this data.


In [67]:
dataset['translation'][0]['de']


'Eine republikanische Strategie, um der Wiederwahl von Obama entgegenzutreten'

In [71]:
en_Sentences = [ i['en'] for i in dataset['translation'] ]
de_Sentences =[ i['de'] for i in dataset['translation'] ]

print(len(en_Sentences))
print(len(de_Sentences))

3000
3000


In [72]:
t0 = time.time()
modelName = "facebook/mbart-large-50-many-to-many-mmt" # "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(modelName)
tokenizer = MBart50TokenizerFast.from_pretrained(modelName)
print(f'time in minutes is : {(time.time() - t0)/60} m')

time in minutes is : 0.44435649712880454 m


In [73]:
print(tokenizer.vocab_size)

250054


## Generating Translation

In [76]:
%%time
t0 = time.time()

generated_translations = []
i=0
for english_sentence in en_Sentences:
    if (i%100==0):
        print(i)
    tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(english_sentence, return_tensors="pt", max_length=1024, truncation=True)
    generated_tokens = model.generate(
        **encoded_en,
        forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"]
    )
    gen_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    generated_translations.append(gen_text)
    i+=1
print(f'time in minutes is : {(time.time() - t0)/60} m')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
time in minutes is : 483.57855308850606 m
CPU times: user 12h 45min 10s, sys: 41.3 s, total: 12h 45min 51s
Wall time: 8h 3min 34s


In [52]:
generated_translations

[['Europäische Arzneimittelagentur'],
 ['EMEA/ H/ C/ 471'],
 ['EUROPEAN PUBLIC ASSESSMENT REPORT (EPAR)'],
 ['ABILIFY'],
 ['EPAR-Zusammenfassung für die Öffentlichkeit']]

In [77]:
generated_translationss = [text[0] for text in generated_translations]


In [83]:
len(generated_translationss)

3000

In [78]:
ref = [i for i in de_Sentences]

In [84]:
len(ref)

3000

In [85]:

bleu = BLEU()

print(f'BLEU score is : {bleu.corpus_score(generated_translationss, [ref])}')

chrf = CHRF()
print(f'chrF2 score is :{chrf.corpus_score(generated_translationss, [ref])}')

BLEU score is : BLEU = 25.25 57.5/31.0/19.2/12.4 (BP = 0.989 ratio = 0.989 hyp_len = 63052 ref_len = 63737)
chrF2 score is :chrF2 = 53.97


## Comet

In [90]:

comt_model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comt_model_path)


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [91]:
data = [
    {
        "src": en_Sentences,
        "mt":  generated_translationss,
        "ref": [ref]
    }
]

In [92]:
# Calculate COMET score
comet_model_output = comet_model.predict(data, batch_size=8, gpus=1)
print(comet_model_output)

Predicting: 0it [00:00, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Predicting DataLoader 0: 100%|██████████| 1/1 [00:07<00:00,  7.94s/it]

Prediction([('scores', [0.9172835946083069]), ('system_score', 0.9172835946083069)])
